# 02 - Tokenización y Creación del Dataset en PyTorch

**Materia:** Redes Neuronales Profundas — UTN FRM

**Objetivo:** Tokenizar las reseñas limpias usando el tokenizer de BERT (`bert-base-uncased`), crear los TensorDatasets y dividir en conjuntos de entrenamiento, validación y test.

---

## ¿Qué es la tokenización en BERT?

BERT no recibe texto crudo como entrada. El texto debe convertirse en una secuencia de tokens numéricos. El tokenizer de BERT realiza:

1. **Tokenización WordPiece:** Divide palabras en sub-palabras. Ej: "playing" → ["play", "##ing"]. Esto permite manejar palabras desconocidas.
2. **Tokens especiales:**
   - `[CLS]` al inicio: su representación final se usa como embedding de la oración completa para clasificación.
   - `[SEP]` al final: marca el fin de la secuencia.
3. **Mapeo a IDs:** Cada token se convierte a su ID numérico en el vocabulario de BERT (30,522 tokens).
4. **Padding:** Se rellena con `[PAD]` (ID=0) hasta alcanzar `max_length`.
5. **Attention Mask:** Vector binario que indica cuáles tokens son reales (1) y cuáles son padding (0).

### Ejemplo visual:
```
Texto:    "great game"
Tokens:   [CLS] great game [SEP] [PAD] [PAD] ...
IDs:      101   2307  2208  102   0     0     ...
Mask:     1     1     1     1     0     0     ...
```

## 1. Importación de Librerías

In [ ]:
import os
import torch
import pandas as pd
import numpy as np
from transformers import BertTokenizer
from torch.utils.data import TensorDataset, random_split

## 2. Configuración

- **MAX_LENGTH = 128:** BERT soporta hasta 512 tokens, pero 128 es suficiente para la mayoría de las reseñas y reduce memoria y tiempo.
- **MODEL_NAME = 'bert-base-uncased':** Modelo BERT base en inglés, sin distinción de mayúsculas.

In [ ]:
CLEAN_DATA_PATH = "../data/clean_reviews.csv"
TENSORS_DIR = "../data/tensors/"
MAX_LENGTH = 128
MODEL_NAME = 'bert-base-uncased'
RANDOM_SEED = 42

## 3. Carga del Dataset Limpio

In [ ]:
df = pd.read_csv(CLEAN_DATA_PATH)
print(f"Dataset cargado: {len(df):,} muestras")
print(f"Distribución: {df['label'].value_counts().to_dict()}")

sentences = df['text'].values
labels = df['label'].values

## 4. Carga del Tokenizer de BERT

Usamos `BertTokenizer.from_pretrained()` para cargar el tokenizer preentrenado. El parámetro `do_lower_case=True` indica que el texto se convertirá a minúsculas (consistente con `bert-base-uncased`).

In [ ]:
print(f"Cargando tokenizer de BERT ({MODEL_NAME})...")
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME, do_lower_case=True)
print(f"Vocabulario: {tokenizer.vocab_size:,} tokens")

### Ejemplo de Tokenización

In [ ]:
ejemplo = sentences[1]
print(f"Texto original: {ejemplo}")
print(f"\nTokens: {tokenizer.tokenize(ejemplo)[:20]}...")

encoded = tokenizer(
    ejemplo,
    add_special_tokens=True,
    max_length=MAX_LENGTH,
    padding='max_length',
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt',
)
print(f"\nInput IDs (primeros 20): {encoded['input_ids'][0][:20].tolist()}")
print(f"Attention Mask (primeros 20): {encoded['attention_mask'][0][:20].tolist()}")
print(f"Longitud total: {encoded['input_ids'].shape[1]} tokens")

## 5. Tokenización de Todo el Dataset

Aplicamos la tokenización a todas las reseñas. Para cada una, el tokenizer:
1. Agrega `[CLS]` y `[SEP]` (`add_special_tokens=True`)
2. Trunca a `max_length` si excede (`truncation=True`)
3. Rellena con `[PAD]` hasta `max_length` (`padding='max_length'`)
4. Genera la attention mask (`return_attention_mask=True`)

El resultado son tres tensores: **input_ids**, **attention_masks** y **labels**.

In [ ]:
input_ids = []
attention_masks = []

print(f"Tokenizando {len(sentences):,} reseñas...")

for i, sent in enumerate(sentences):
    if (i + 1) % 10000 == 0:
        print(f"  Procesadas {i+1:,} / {len(sentences):,}")

    encoded_dict = tokenizer(
        sent,
        add_special_tokens=True,
        max_length=MAX_LENGTH,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt',
    )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

# Concatenar en tensores únicos
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels_tensor = torch.tensor(labels, dtype=torch.long)

print(f"\nTokenización completa.")
print(f"  input_ids shape:      {input_ids.shape}")
print(f"  attention_masks shape: {attention_masks.shape}")
print(f"  labels shape:          {labels_tensor.shape}")

## 6. División en Train / Validation / Test

- **Entrenamiento (80%):** Para ajustar los pesos del modelo.
- **Validación (10%):** Para monitorear rendimiento y evitar overfitting.
- **Test (10%):** Evaluación final (nunca visto durante el entrenamiento).

Usamos `TensorDataset` y `random_split` de PyTorch, como en el notebook de clase.

In [ ]:
dataset = TensorDataset(input_ids, attention_masks, labels_tensor)

total = len(dataset)
train_size = int(0.8 * total)
val_size = int(0.1 * total)
test_size = total - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(
    dataset, [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(RANDOM_SEED)
)

print(f"Entrenamiento: {train_size:,} ({train_size/total:.0%})")
print(f"Validación:    {val_size:,} ({val_size/total:.0%})")
print(f"Test:          {test_size:,} ({test_size/total:.0%})")

## 7. Guardado de los Datasets

Guardamos en dos formatos:
1. **Tensores `.pt`:** Para cargar directamente en entrenamiento con `torch.load()`.
2. **Archivos `.csv`:** Documentación de los splits con el texto original.

In [ ]:
os.makedirs(TENSORS_DIR, exist_ok=True)
torch.save(train_dataset, os.path.join(TENSORS_DIR, "train_dataset.pt"))
torch.save(val_dataset, os.path.join(TENSORS_DIR, "val_dataset.pt"))
torch.save(test_dataset, os.path.join(TENSORS_DIR, "test_dataset.pt"))
print(f"Datasets .pt guardados en: {TENSORS_DIR}")

In [ ]:
data_dir = os.path.dirname(CLEAN_DATA_PATH)
df.iloc[train_dataset.indices].reset_index(drop=True).to_csv(os.path.join(data_dir, "training_data.csv"), index=False)
df.iloc[val_dataset.indices].reset_index(drop=True).to_csv(os.path.join(data_dir, "validation_data.csv"), index=False)
df.iloc[test_dataset.indices].reset_index(drop=True).to_csv(os.path.join(data_dir, "test_data.csv"), index=False)
print(f"CSVs guardados en: {data_dir}")
print(f"  training_data.csv   ({len(train_dataset.indices):,} filas)")
print(f"  validation_data.csv ({len(val_dataset.indices):,} filas)")
print(f"  test_data.csv       ({len(test_dataset.indices):,} filas)")

## Resumen

1. Cargamos las 50,000 reseñas limpias.
2. Tokenizamos con `BertTokenizer`: tokens especiales `[CLS]`/`[SEP]`, padding a 128, attention masks.
3. Creamos `TensorDataset` con (input_ids, attention_masks, labels).
4. Dividimos en Train (80%), Validation (10%) y Test (10%).
5. Guardamos como `.pt` y `.csv`.

**Siguiente paso:** Fine-tuning del modelo BERT.